# Example of recipe computation, model fit, predict* and conversion to raster using API of antares3 and [kale](https://github.com/kubeflow-kale/kale) functionality

*Prediction is pixel wise.

**Will use an already ingested and processed Landsat8 data via antares3**

## Some imports

In [1]:
import sys

import os
import json

import matplotlib
from matplotlib.patches import Patch
from matplotlib import pyplot as plt
import numpy as np
from shapely.geometry import Point
import rasterio
import dill
import geopandas as gpd
import fiona
from affine import Affine
from rasterio.features import rasterize

from datacube.api import GridWorkflow

from madmex.util.db import get_cmap_from_scheme
from madmex.models import Tag
from madmex.overlay.extractions import zonal_stats_xarray
from madmex.io.vector_db import VectorDb
from madmex.wrappers import gwf_query
from madmex.modeling.supervised.xgb import Model
from madmex.models import Tag
from madmex.overlay.extractions import zonal_stats_xarray
from madmex.util import randomword
from django.contrib.gis.geos.geometry import GEOSGeometry
from madmex.models import PredictObject

## Recipe computation

In [ ]:
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "true")
sys.__stdout__ = sys.stdout
sys.stdout = sys.__stdout__
name_of_product = 'recipe_chiapas_L8_17'

In [ ]:
region = 'Chiapas'
name_of_product = 'ls8_espa_mexico'
begin = '2017-01-01'
end = '2017-12-31'
gwf_kwargs = {'region': region, 
              'product': name_of_product, 
              'begin': begin, 
              'end':end}
iterable = gwf_query(**gwf_kwargs)

#query

list_iter = list(iterable)

list_iter_sorted = sorted(list_iter, key = lambda x: (x[0][0], x[0][1]))



In [ ]:
print(list_iter_sorted)

In [ ]:
# Select datacube tile index: (59, -31)

tile = [index for index in list_iter_sorted if index[0] == (54, -38)][0]

# Load via Grid Workflow API
ds = GridWorkflow.load(tile[1])

## Fit model